In [1]:
# Load all the required libraries
import json
import pandas as pd

In [2]:
# Load different user schemas as json dictionaries
with open('./4me/users.schema.json') as file:
    users_4me = json.load(file)
with open('./scim-dev/users.json') as file:
    users_scim_div = json.load(file)
with open('./SF/users.json') as file:
    users_sf = json.load(file)
# with open('./Slack/slack-users-schemas.json') as file:
#     users_slack = json.load(file)

In [3]:
users_4me_attribute_list = users_4me['attributes']
users_scim_div_attribute_list = users_scim_div['attributes']
users_sf_attribute_list = users_sf["attributes"]
# users_slack_attribute_list = users_slack['attributes']


In [4]:
def get_attribute_name(attribute_list):
    attribute_name_list = []
    for attribute in attribute_list:
        attribute_name_list.append(attribute['name'])
    return sorted(attribute_name_list)

In [5]:
users_4me_attribute_name_list = get_attribute_name(users_4me_attribute_list)
users_scim_div_attribute_name_list = get_attribute_name(users_scim_div_attribute_list)
users_sf_attribute_name_list = get_attribute_name(users_sf_attribute_list)


In [6]:
pd.DataFrame(data=[users_4me_attribute_name_list,users_scim_div_attribute_name_list,users_sf_attribute_name_list], index=["4me", "scim.dev", "salesforce"]).transpose()

,4me,scim.dev,salesforce
0,active,active,active
1,addresses,addresses,addresses
2,displayName,displayName,displayName
3,emails,emails,emailEncodingKey
4,entitlements,entitlements,emails
5,groups,groups,entitlements
6,ims,ims,externalId
7,locale,locale,groups
8,name,name,id
9,nickName,nickName,locale


In [7]:
df_4me = pd.DataFrame(data=users_4me_attribute_name_list, columns=["4me"])
df_scim_div = pd.DataFrame(data=users_scim_div_attribute_name_list, columns=["scim.dev"])
df_sf = pd.DataFrame(data=users_sf_attribute_name_list, columns=["salesforce"])

df_4me.merge(df_scim_div, how="outer", left_on="4me", right_on="scim.dev").merge(df_sf, how="outer", left_on="4me", right_on="salesforce")
df_merged = df_4me.merge(df_scim_div, how="outer", left_on="4me", right_on="scim.dev").merge(df_sf, how="outer", left_on="4me", right_on="salesforce")
df_merged.style.highlight_null(color='red')


,4me,scim.dev,salesforce
0,active,active,active
1,addresses,addresses,addresses
2,displayName,displayName,displayName
3,nan,nan,emailEncodingKey
4,emails,emails,emails
5,entitlements,entitlements,entitlements
6,nan,nan,externalId
7,groups,groups,groups
8,nan,nan,id
9,ims,ims,nan


In [8]:
common_elements = set(users_4me_attribute_name_list) & set(users_scim_div_attribute_name_list) & set(users_sf_attribute_name_list)
common_elements = sorted(common_elements)

for common_element in common_elements:
    for attribute in users_4me_attribute_list:
        if attribute['name'] == common_element:
            users_4me_attribute = attribute
    for attribute in users_scim_div_attribute_list:
        if attribute['name'] == common_element:
            users_scim_div_attribute = attribute
    for attribute in users_sf_attribute_list:
        if attribute['name'] == common_element:
            users_sf_attribute = attribute

    print(f"Attribute: {common_element}")
    
    df_4me = pd.DataFrame({'attribute': users_4me_attribute.keys(), 'value': users_4me_attribute.values()})
    df_scim_div = pd.DataFrame({'attribute': users_scim_div_attribute.keys(), 'value': users_scim_div_attribute.values()})
    df_sf = pd.DataFrame({'attribute': users_sf_attribute.keys(), 'value': users_sf_attribute.values()})
    df = df_4me.merge(df_scim_div, how="outer", on="attribute", suffixes=("_4me", "_scim.dev")).merge(df_sf, how="outer", on="attribute", suffixes=("_salesforce", "__"))
    print(df.to_markdown())
    

Attribute: active
|    | attribute   | value_4me                                                    | value_scim.dev                                               | value                                       |
|---:|:------------|:-------------------------------------------------------------|:-------------------------------------------------------------|:--------------------------------------------|
|  0 | caseExact   | nan                                                          | False                                                        | False                                       |
|  1 | description | A Boolean value indicating the User's administrative status. | A Boolean value indicating the User's administrative status. | A Boolean value indicating the Users status |
|  2 | multiValued | False                                                        | False                                                        | False                                       |
|  3 | mutability